In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import csv
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# download merged_all.csv
!gdown 1eDOBqIQJRLThxFJ2-pg84SfbqlyQWWpp --quiet

# download utility.py
!wget https://raw.githubusercontent.com/lashhw/AI-Final-Project/main/azure-functions/models/common/utility.py --quiet

# download ids_train.npy, ids_train.npy 是一個array，包含250個停車場的ID
!wget https://raw.githubusercontent.com/lashhw/AI-Final-Project/main/azure-functions/models/common/files/ids_train.npy --quiet

In [5]:
# 讀取 merged_all.csv, 存成data_df

data_df = pd.read_csv('merged_all.csv')
data_df = data_df.set_index('ID').T

# convert to UTC+0
new_index = list(pd.to_datetime(data_df.index))
for i in range(len(new_index)):
    new_index[i] = new_index[i] - timedelta(hours=8)
data_df.index = new_index

ids_train = np.load('ids_train.npy')
data_df = data_df.loc[:, ids_train]

data_df

ID,10001,10003,10004,10008,10009,10010,10011,10013,10014,10015,...,180005,200001,210001,210002,230001,230008,230018,260005,270008,290002
2022-03-20 02:42:00,66.0,4.0,41.0,65.0,37.0,24.0,45.0,390.0,3.0,17.0,...,79.0,34.0,23.0,90.0,12.0,0.0,113.0,42.0,6.0,120.0
2022-03-20 02:45:00,67.0,3.0,42.0,62.0,37.0,24.0,46.0,387.0,4.0,17.0,...,69.0,34.0,20.0,91.0,11.0,1.0,104.0,42.0,11.0,105.0
2022-03-20 02:48:00,68.0,2.0,41.0,60.0,35.0,25.0,46.0,384.0,6.0,17.0,...,59.0,33.0,21.0,92.0,12.0,0.0,104.0,42.0,7.0,103.0
2022-03-20 02:51:00,67.0,1.0,40.0,58.0,35.0,35.0,46.0,382.0,5.0,17.0,...,57.0,33.0,23.0,93.0,11.0,0.0,95.0,42.0,6.0,102.0
2022-03-20 02:54:00,66.0,1.0,43.0,56.0,33.0,45.0,48.0,380.0,6.0,17.0,...,52.0,32.0,23.0,92.0,11.0,0.0,78.0,42.0,8.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-05 15:45:00,22.0,10.0,67.0,106.0,66.0,16.0,22.0,397.0,44.0,29.0,...,91.0,38.0,20.0,49.0,29.0,117.0,159.0,27.0,20.0,260.0
2022-06-05 15:48:00,21.0,10.0,67.0,105.0,66.0,16.0,22.0,397.0,44.0,29.0,...,91.0,38.0,21.0,48.0,29.0,117.0,159.0,27.0,20.0,260.0
2022-06-05 15:51:00,21.0,10.0,67.0,105.0,67.0,16.0,22.0,397.0,45.0,29.0,...,91.0,38.0,21.0,48.0,29.0,117.0,159.0,27.0,20.0,260.0
2022-06-05 15:54:00,21.0,10.0,67.0,105.0,68.0,16.0,22.0,397.0,45.0,29.0,...,91.0,38.0,21.0,48.0,29.0,117.0,159.0,27.0,20.0,260.0


In [6]:
# 前處理

from utility import preprocess
data_df, data_smooth_df = preprocess(data_df)

In [7]:
cars = data_smooth_df.T
d = datetime(2022,3,20,2,15)
interval = timedelta(minutes=15)
idx = 0
while(d != datetime(2022,5,23)):
  d += interval
  idx += 1
print(idx)

6135


In [8]:
cars.fillna(value=-9, inplace=True)

In [9]:
# idx = 0
# for x in cars:
#   # print(x)
#   if x == '2022-06-05 15:30:00':
#     print(idx)
#   idx += 1
cars.shape

(250, 7446)

In [17]:
from statsmodels.tsa.ar_model import AR

MAE = []
MSE = []
scaled_MAE = []
scaled_MSE = []
for idx in range(250):
  print(idx)
  startindex = 6135
  start = datetime(2022 , 5 , 30 , 0 , 0)
  lot_max = 0
  abss = []
  abssquare = []
  while(1):
    v = cars.values
    if(start > datetime(2022 , 6 , 2 , 00 , 00)):
      break
    try:
      interval = timedelta(minutes=15)
      ttlplus = int(1440/15*3-1)

      d = start
      train_t = []
      for i in range(ttlplus):
        train_t.append(d)
        d += interval

      train_d = v[idx][startindex:startindex+ttlplus]

      startindex = startindex+1
      interval_ = timedelta(minutes=15)
      start = start + interval_

      model_fit = AR(train_d,train_t,interval).fit()
      # p = model_fit.k_ar
      # print(p)

      predict_start = train_t[-1] + interval
      interval__ = timedelta(minutes = 1440-15)
      predict_end = predict_start+interval__
      res = model_fit.predict(start=predict_start,end=predict_end)
      # print(len(res))
      # print(cars.values[2][31185])

      d = predict_start
      test_t = []
      for i in range(96):
        test_t.append(d)
        d += interval
      test_d = v[idx][startindex+ttlplus:startindex+ttlplus + 96]

      # plt.figure(figsize=(10,4))
      # plt.plot(test_t,res,color='r')
      # plt.plot(test_t,test_d)
      # plt.plot(train_t,train_d)     
      # plt.show()
      # print(idx)
      abss.append(np.mean(abs(np.array(res) - np.array(test_d))))
      abssquare.append(np.mean((np.array(res) - np.array(test_d))**2))
      lot_max = max(lot_max, max(test_d))
    except:
      continue
  if(len(abss) != 0 and len(abssquare) !=0 and lot_max > 0):
    MAE.append(np.mean(np.array(abss)))
    MSE.append(np.mean(np.array(abssquare)))
    scaled_MAE.append(np.mean(np.array(abss)/lot_max))
    scaled_MSE.append(np.mean(np.array(abssquare)/lot_max))
    print(MAE[-1],MSE[-1],scaled_MAE[-1],scaled_MSE[-1])

MAE.sort()
MSE.sort()
scaled_MAE.sort()
scaled_MSE.sort()
for i in range(10):
  MAE.pop()
  MSE.pop()
  scaled_MAE.pop()
  scaled_MSE.pop()

if(len(MAE) != 0 and len(MSE) != 0 and len(scaled_MAE) != 0 and len(scaled_MSE) != 0):
  print('--------------------------------')
  MAE = np.mean(np.array(MAE))
  MSE = np.mean(np.array(MSE))
  scaled_MAE = np.mean(np.array(scaled_MAE))
  scaled_MSE = np.mean(np.array(scaled_MSE))
  print(MAE,MSE,scaled_MAE,scaled_MSE)
  print('--------------------------------')

0
8.02943013267108 103.74884308152838 0.10500104789683642 1.3567260766513454
1
3.022057452608456 12.589595279145525 0.20798743651813179 0.866455284180697
2
10.247030415650157 170.222212234311 0.15514050591446113 2.577172024743543
3
14.762282501832226 356.86763428728284 0.1230190208486019 2.973896952394024
4
30586.470588005734 831846940850.377 427.84264355861984 11635850340.612349
5
17.497611070892706 456.3501854333678 0.23389401244342609 6.100122783496429
6
10.817039496566615 192.75914230116965 0.20635329066323188 3.677206072132195
7
84.70994456035022 10074.597869397485 0.20944478813289708 24.90937784496844
8
9.27200249847161 130.34916791343807 0.20077961235321806 2.822632479719317
9
6.011585799307024 50.62080709254217 0.1939221225582911 1.6329292610497477
10
11
10.63906783876489 154.727482552855 0.2309828015363632 3.359259282519648
12
13
19.21428773437816 656.8995013242964 0.12096630404418385 4.135605019669455
14
1.7708618681242565 4.983694568232467 0.14757182234368804 0.4153078806860